In [4]:
import pandas as pd
import numpy as np
import src.functions as src
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

features = "dataset2_features.csv"
features = pd.read_csv(features, sep=',', header=0, index_col=None)
Y = features[['labels']]
# feat_not_theta = [i for i in features.columns if "Theta" not in i]
# X = features.drop(feat_not_theta, axis=1)
# not_time_feat = [i for i in features.columns if ("ERN" not in i) & ("Pe" not in i) & ("amp" not in i)]
# X = features.drop(not_time_feat, axis=1)

#not_freq_feat = [i for i in features.columns if ("Delta" not in i) & ("Theta" not in i) & ("Alpha" not in i) &
 #                ("Freq" not in i)]
#X = features.drop(not_freq_feat, axis=1)
# X = features.drop('Participant', axis=1)
X = features.drop('labels', axis=1)
print(len(X.columns)) 


1629


In [2]:
# Remove features with coefficient of variation < 0.2
variance = X.std()/X.mean()
low_variance = [i for i in variance.index if variance[i] < 0.2]
X1 = X.drop(low_variance, axis=1)
print(len(X1.columns))

y0 = Y[Y['labels'] == 0]
feat_corr = np.zeros(len(X1.columns))
feat_ttest = np.zeros(len(X1.columns))

Y = np.ravel(Y)
i = 0
splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
for train_index, test_index in sss.split(X1, Y):
    X_train, X_test = X1.iloc[train_index, :], X1.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]

    ind_y0 = [j for j in range(0, len(X_train)) if X_train.index[j] in y0.index]
    ind_y1 = [j for j in range(0, len(X_train)) if X_train.index[j] not in y0.index]
    X_ind_y0 = X_train.iloc[ind_y0, :]
    X_ind_y1 = X_train.iloc[ind_y1, :]

    corr = []

    # Remove correlated features
    for col1 in range(0, len(X1.columns)-1):
        for col2 in range(col1+1, len(X1.columns)):
            if abs(np.corrcoef(X_train[X1.columns[col1]], X_train[X1.columns[col2]])[0, 1]) > 0.9:
                if stats.ttest_ind(X_ind_y0[X1.columns[col1]], X_ind_y1[X1.columns[col1]]).pvalue < \
                        stats.ttest_ind(X_ind_y0[X1.columns[col2]], X_ind_y1[X1.columns[col2]]).pvalue:
                    if col2 not in corr:
                        feat_corr[col2] += 1
                        corr.append(col2)
                    elif col1 not in corr:
                        feat_corr[col1] += 1
                        corr.append(col1)

    # Relevance: t-test between independent variables and output
    ttest = [stats.ttest_ind(X_ind_y0[col], X_ind_y1[col])[1] for col in X1.columns]
    ttest_order = np.argsort(ttest)
    feat_ttest += np.argsort(ttest_order)  # min(p-value) -> +0, max(p-value) -> +len(X.columns)-1

    i += 1

print(len(list(X1.columns[feat_corr > splits/2])))
X1 = X1.drop(list(X1.columns[feat_corr > splits/2]), axis=1)  # Remove correlated features
feat_ttest = feat_ttest[feat_corr <= splits/2]

X1 = X1.drop(X1.columns[np.argsort(np.argsort(feat_ttest)) >= 40], axis=1)
feat_ttest = feat_ttest[np.argsort(np.argsort(feat_ttest)) < 40]
Xcol_sort = [x for _, x in sorted(zip(feat_ttest, X1.columns))]
print(Xcol_sort)

1484
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
29
['1_0', '7_0', '8_F1', '12_F1', '8_F3', '8_FZ', '12_FZ', '12_AF3', '8_FC1', '8_F2', '8_AF3', '12_F3', '21_F1', '17_FT7', '33_CZ', '21_FC1', '8_FC2', '21_FCZ', '21_F3', '8_F4', '21_FZ', '21_CZ', '12_FC1', '12_F4', '12_F2', '35_CZ', '8_FC3', '8_POZ', '35_CPZ', '35_C1', '17_F1', '20_PO4', '8_TP8', '18_FT7', '8_AF4', '21_F2', '12_FC3', '8_OZ', '10_P4', '27_P4']


In [5]:
f1score = []
precision = []
recall = []
specificity = []
npv = []
bal_acc = []

splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
svm = SVC(class_weight='balanced')
scaler = preprocessing.StandardScaler()
X_col = X.columns
Y=np.array(Y)

for train_index, test_index in sss.split(X, Y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]
    scal = scaler.fit(X_train)
    X_train = scal.transform(X_train)  # Variables standardization
    X_test = scal.transform(X_test)  # Variables standardization
    X_train = pd.DataFrame(X_train, columns=X_col)
    X_test = pd.DataFrame(X_test, columns=X_col)
    clf = svm.fit(X_train, Y_train)
    Y_predicted = clf.predict(X_test)
    f1score.append(f1_score(Y_test, Y_predicted))
    precision.append(precision_score(Y_test, Y_predicted))  # Precision = Positive predictive value
    npv.append(precision_score(Y_test, Y_predicted, pos_label=0))  # Negative predictive value
    recall.append(recall_score(Y_test, Y_predicted))  # Recall = Sensitivity
    specificity.append(recall_score(Y_test, Y_predicted, pos_label=0))
    bal_acc.append(balanced_accuracy_score(Y_test, Y_predicted))

# print(np.mean(f1score))
# print(np.mean(precision))
print(np.mean(recall))
print(np.std(recall))
print(np.mean(specificity))
print(np.std(specificity))
# print(np.mean(npv))
print(np.mean(bal_acc))
print(np.std(bal_acc))

0.5740753424657534
0.027829025046090826
0.7290807017543861
0.014642915377759783
0.6515780221100697
0.013140676809983404


In [5]:
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=X, y=Y, K=30)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X[i]], axis=1)
X=combined_df

100%|██████████| 30/30 [00:13<00:00,  2.25it/s]


In [6]:
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X[i]], axis=1)
X=combined_df

In [7]:
src.classification(X,Y)

Mean Sensitivity: 0.5965
Std Sensitivity: 0.0255
Mean Specificity: 0.7732
Std Specificity: 0.0122
Mean Balanced Accuracy: 0.6848
Std Balanced Accuracy: 0.0123


In [8]:
bal_acc #teste de proporções, testar quao acima do chance level está
#ROC curve
#R^2
#

[0.6933417447728911,
 0.6730749819754867,
 0.6853977409276617,
 0.7070127373227589,
 0.6908014900264359,
 0.6512496995914444,
 0.6919802931987503,
 0.6797837058399423,
 0.707223023311704,
 0.7045001201634222,
 0.6688920932468156,
 0.6923876472001923,
 0.7086265320836338,
 0.7024789714011055,
 0.6942261475606826,
 0.6948437875510695,
 0.6752223023311704,
 0.690549146839702,
 0.689931506849315,
 0.6870824321076665,
 0.6794748858447488,
 0.6565693342946407,
 0.6806536890170631,
 0.69014179283826,
 0.6540422975246336,
 0.6932011535688536,
 0.686240086517664,
 0.6760081711127133,
 0.6919382360009613,
 0.6893559240567171,
 0.6829836577745734,
 0.6752643595289594,
 0.6801345830329248,
 0.6892573900504686,
 0.7050192261475607,
 0.6891732756548907,
 0.6766258111031003,
 0.6772855082912761,
 0.6834751261715933,
 0.6809769286229272,
 0.6781134342706081,
 0.6784643114635904,
 0.6675306416726748,
 0.6853556837298727,
 0.7003737082432108,
 0.6917700072098054,
 0.6921353040134584,
 0.6940014419610671

0.5771232876712328
0.026507291773589414
0.7865192982456141
0.012696245776507414
0.6818212929584234
0.01276152874444839